In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
response = requests.get('https://emagazine.com/archive/')
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
articles = soup.find_all('article')

In [4]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
referer = 'https://www.resilience.org/latest-articles/'
headers = {'User-Agent': user_agent, 'referer':referer}

In [29]:
titles =[]
links = []

for page in range(1, 9):
    
    response = requests.get('https://emagazine.com/archive/page/{}/'.format(page), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = soup.find_all('article')
    
    for article in articles:
    
        title_link = article.find('h2', {'class':'post-title entry-title'})
        titles.append(title_link.text)
        links.append(title_link.find('a')['href'])

In [30]:
emag = pd.DataFrame({'title':titles, 'link':links})

In [ ]:
emag.head(

In [ ]:
t0 = time.time()
time.sleep(4)
print(time.time() - t0 + 60)

In [10]:
import ast

In [31]:
link_attributes2 = {}
t0 = time.time()
for i in range(0, len(emag.link)):
    
    if i % 200 == 0:
        print(i, (time.time() - t0) / 60)

    link = emag.link.iloc[i]
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    

    footer = soup.find_all('script', {'type':'application/ld+json'})[-1].text
    f = ast.literal_eval(footer)
    try:
        date = f[0]['datePublished'][:10]
    except:
        print(link, "no date")
        date = ''
    
    try:
        author = f[0]['author']['name']
    except:
        print(link, "no author")
        author = ''
    
    text = " ".join([p.text for p in soup.find('div', {'class':'entry-content'}).find_all('p')])
    text = text.replace('\xa0', '')
    
    attributes = {'link':link, 'date':date, 'author':author, 'text':text}
    link_attributes2[i] = attributes

0 7.907549540201822e-06


In [13]:
len(link_attributes2)

70

In [32]:
emag['link'] = [link_attributes2[i]['link'] for i in range(80)]
emag['date'] = [link_attributes2[i]['date'] for i in range(80)]
emag['author'] = [link_attributes2[i]['author'] for i in range(80)]
emag['text'] = [link_attributes2[i]['text'] for i in range(80)]

In [28]:
emag_new = pd.DataFrame({'title': [link_attributes2[i]['title'] for i in range(80)],
                      'link':   [link_attributes2[i]['link'] for i in range(80)],
                      'date':   [link_attributes2[i]['date'] for i in range(80)],
                      'author': [link_attributes2[i]['author'] for i in range(80)],
                      'text':   [link_attributes2[i]['text'] for i in range(80)]})

KeyError: 'title'

In [ ]:
emag2 = pd.DataFrame({'link':   [link_attributes[i]['link'] for i in range(3304)],
                      'date':   [link_attributes[i]['date'] for i in range(3304)],
                      'author': [link_attributes[i]['author'] for i in range(3304)],
                      'text':   [link_attributes[i]['text'] for i in range(3304)]})

emag3 = pd.DataFrame({'link':   [link_attributes2[i]['link'] for i in range(3304, 7915)],
                      'date':   [link_attributes2[i]['date'] for i in range(3304, 7915)],
                      'author': [link_attributes2[i]['author'] for i in range(3304, 7915)],
                      'text':   [link_attributes2[i]['text'] for i in range(3304, 7915)]})

In [ ]:
emag.to_pickle('../Data/Emagazine/emag.pkl')
emag2.to_pickle('../Data/Emagazine/emag2.pkl')#
emag3.to_pickle('../Data/Emagazine/emag3.pkl')

In [33]:
emag  = pd.read_pickle('../Data/Emagazine/emag.pkl')
emag2 = pd.read_pickle('../Data/Emagazine/emag2.pkl')
emag3 = pd.read_pickle('../Data/Emagazine/emag3.pkl')
text = pd.read_pickle('../Data/Emagazine/emag4.pkl')

In [34]:
date_author = pd.concat([emag2, emag3])[['link', 'date', 'author']].reset_index()
date_author['index'] = date_author.index
emag['index'] = emag.reset_index().index
text['index'] = text.reset_index().index

In [35]:
emag = emag.merge(date_author, on=['link', 'index'], how='left')
emag = emag.merge(text, on=['link', 'index'], how='left')

In [37]:
emag.to_pickle('../Data/Emagazine/emag_final.pkl')

In [4]:
emag2.head()

,link,date,author,text
0,https://emagazine.com/roof-replacement/,2021-12-05,E - The Environmental Magazine,"Imagine stretching out on the couch, a World ..."
1,https://emagazine.com/homesteading-safely/,2021-12-05,E - The Environmental Magazine,"Imagine stretching out on the couch, a World ..."
2,https://emagazine.com/why-its-important-to-use...,2021-12-05,E - The Environmental Magazine,"Imagine stretching out on the couch, a World ..."
3,https://emagazine.com/nornickel-lifts-up-inves...,2021-12-02,E - The Environmental Magazine,"Imagine stretching out on the couch, a World ..."
4,https://emagazine.com/ocean-winds-bringing-us-...,2021-12-02,H. Bruce Franklin,"Imagine stretching out on the couch, a World ..."


In [27]:
emag_new

,link,date,author,text
0,https://emagazine.com/how-to-become-a-sustaina...,2022-02-05,E - The Environmental Magazine,Are you looking for different influence option...
1,https://emagazine.com/lithium-battery-waste/,2022-02-03,Lindsey Blomberg,"—W. Alexander, San Francisco, CA Record-breaki..."
2,https://emagazine.com/make-your-business-more-...,2022-02-02,E - The Environmental Magazine,Sustainability is key for the future of our pl...
3,https://emagazine.com/what-gear-should-you-hav...,2022-02-02,E - The Environmental Magazine,Are you planning to visit the wilderness? Whe...
4,https://emagazine.com/easy-and-eco-friendly-wa...,2022-02-02,E - The Environmental Magazine,The process of renovating a home to increase i...
...,...,...,...,...
75,https://emagazine.com/the-top-4-reasons-to-buy...,2021-12-10,E - The Environmental Magazine,"A dog crate is an enclosure made of plastic, w..."
76,https://emagazine.com/lessons-learned_montreal...,2021-12-09,Roddy Scheer and Doug Moss,"—C. Marin, St. Louis, MO 1987’s Montreal Proto..."
77,https://emagazine.com/youth-climate-activists/,2021-12-09,Jane Marsh,Whitney Houston once sang that the children ar...
78,https://emagazine.com/life-in-plastic-new-book...,2021-12-09,E - The Environmental Magazine,The essays in the new book Life in Plastic: Ar...


In [33]:
e = pd.read_pickle('../Data/Emagazine/emag_final.pkl')

In [34]:
emag_new = pd.concat([e, emag])

In [36]:
emag_new = emag_new[['title', 'link', 'date', 'author', 'text']]

In [37]:
emag_new.to_pickle('../Data/Emagazine/emag_final.pkl')

In [ ]:
e

In [ ]:
list(emag.link).index(link)

In [ ]:
sample_link = emag.link.iloc[7000]

In [ ]:
roof = requests.get(sample_link, headers=headers)
roof_soup = BeautifulSoup(roof.text, 'html.parser')

In [ ]:
footer = roof_soup.find_all('script', {'type':'application/ld+json'})[-1].text

In [ ]:
f = ast.literal_eval(footer)
date = f[0]['datePublished'][:10]
author = f[0]['author']['name']

In [ ]:
text = " ".join([p.text for p in roof_soup.find('div', {'class':'entry-content'}).find_all('p')])
text = text.replace('\xa0', '')

In [ ]:
print(author)
print(date)
print(text)

In [ ]:
sample_link

In [ ]:
import json

In [ ]:
import ast

In [ ]:
footer_list = footer.strip('][').split(',')

In [ ]:
footer_list

In [ ]:
footer_dict = footer[1:footer.index('}')+1]

In [ ]:
footer_dict

#### sum(articles_per_page.values())

In [ ]:
len(titles_links)

In [ ]:
for page in range(11, 15):
    response = requests.get('https://emagazine.com/archive/page/{}/'.format(page), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = soup.find_all('article')
    print(len(articles))

In [ ]:
soup

In [ ]:
len(titles_links)

In [ ]:
articles

Plot articles/words over time

In [5]:
e = pd.read_pickle('../Data/Emagazine/emag_final.pkl')

In [7]:
e['wordcount'] = e.apply(lambda x: len(x.text.split()), axis=1)

In [9]:
e.date

0       2021-12-05
1       2021-12-05
2       2021-12-05
3       2021-12-02
4       2021-12-02
           ...    
7910    2003-11-01
7911    1997-07-01
7912    1997-07-01
7913    1997-03-01
7914    1994-09-01
Name: date, Length: 7915, dtype: object

In [16]:
e['date'] = e.date.apply(lambda x: pd.to_datetime(x))
e['month'] = e.date.apply(lambda x: x.month)
e['year'] = e.date.apply(lambda x: x.year)

In [18]:
toplot = e.groupby(['year', 'month']).agg({'title':'count', 'wordcount':'mean', 'date':'min'})

In [19]:
import plotly.express as px

In [27]:
fig = px.scatter(toplot[toplot.date.dt.year >= 2005], x='date', y='title', size='wordcount',
           labels={'date':'Date', 'title':'Number of Articles'},
           title = 'Figure 4')
fig.update_layout(
    font_family='Times New Roman',
    font_color='black',
    yaxis_range=[0, 200]
)


In [28]:
len(e)

7915